# Prepare Datasets for Deep Autoencoding
## Create database with resized images
#### imports and loading dataset index


In [6]:
import os
import PIL
import matplotlib.pyplot as plt
%matplotlib inline

flickr_index = np.genfromtxt('../flickr_index.txt', dtype = 'str')

#### 128x128px

In [11]:
flickr_images = [file for file in os.listdir('../mirflickr25k/mirflickr/') if file.endswith('jpg')]
for image in flickr_index:
    im = PIL.Image.open(
        os.path.join('../mirflickr25k/mirflickr/', image))
    im = im.resize((128, 128))
    im.save(os.path.join('../flickr_128x128', image), format='png')

#### 32x32px

In [7]:
for image in flickr_index:
    im = PIL.Image.open(
        os.path.join('../deep_learning_data/mirflickr/', image))
    im = im.resize((32, 32))
    im.save(os.path.join('../deep_learning_data/flickr_32x32', image), format='png')

#### create hdf5 files

In [8]:
import h5py
import skimage.io

image_paths_32px = ['../deep_learning_data/flickr_32x32/' + x for x in flickr_index]

with h5py.File('../deep_learning_data/flickr_32x32_2.hdf5') as f:
    X = f.create_dataset(name = 'X', shape=(25000, 32*32*3))
    for i in range(25000):
        X[i] = skimage.io.imread(image_paths_32px[i]).reshape(32*32*3)
        
#image_paths_128px = os.listdir('../flickr_128x128/')
#image_paths_128px = ['../deflickr_128x128/' + x for x in flickr_index]
#with h5py.File('../flickr_128x128.hdf5_2') as f:
#    X = f.create_dataset(name = 'X', shape=(25000,128*128*3))
#    for i in range(25000):
#        X[i] = skimage.io.imread(image_paths_128px[i]).reshape(128*128*3)

### variable resolution retina patches
these patches are of the form:
    ########
    ########
    ##xxxx##
    ##xxxx##
    ##xxxx##
    ##xxxx##
    ########
    ########
where each x are 2x2 pixels and the # are one pixel each 

In [ ]:
import h5py
from skimage.transform import resize
from skimage.io import imread
import numpy as np

image_list = np.genfromtxt('../flickr_index.txt', dtype = 'str')
index = 0

with h5py.File('../deep_learning_data/flickr_32x32_varpatches.hdf5') as f:
    X = f.create_dataset(name = 'X', shape=(25000*9*9,112*3) ) 
    for patch_index_upper in range(0,17,2):
        print('processing patch y pos {} / 17'.format(patch_index_upper))
        for patch_index_left in range(0,17,2):
            for i in range(25000):                    
                img = resize(imread('../deep_learning_data/mirflickr/' + image_list[i]), (32,32))             
                patch = img[patch_index_upper:patch_index_upper+16,patch_index_left:patch_index_left+16,:]
                patch_inner = patch[4:12,4:12,:].reshape(64*3)
                patch_low_res = resize(patch, (8,8))
                patch_outer = np.hstack((
                    patch_low_res[0:2,:,:].reshape(2*8*3),
                    np.hstack(
                            (patch_low_res[3:7,0:2,:], patch_low_res[3:7,6:8,:]) 
                            ).reshape(4*4*3),
                    patch_low_res[6:8,:,:].reshape(2*8*3) )
                                        )
                patch_var_res = np.hstack((patch_inner, patch_outer))
                X[index] = patch_var_res
                index += 1